In [7]:
import tensorflow.keras
from tensorflow.keras.models import model_from_json
import numpy as np
import cv2
import serial
import serial.tools.list_ports
import time

In [8]:
# Read Label file to see how many labels
# --------------------------------------
labels = []
f = open("labels.txt", "r")

for x in f:
  labels.append(x.rstrip('\n'))


label_count = len(labels)

print('labels = ', labels, 'no. of labels = ', label_count, '\r\n')
f.close()

# Load the model
# --------------
model = tensorflow.keras.models.load_model('keras_model.h5')

labels =  ['0 black', '1 pink', '2 none'] no. of labels =  3 



In [ ]:
import serial

ser = serial.Serial('COM22',9600, timeout=1)


while True:
    print("insert op :", end=' ')
    op = input()
    ser.write(op.encode())

insert op : a
insert op : b
insert op : 

In [12]:
import serial

ser = serial.Serial('COM22',9600, timeout=1)

MODE_PAUSE = -1 
MODE_START = 0
mode_status = MODE_PAUSE

video_capture = cv2.VideoCapture(0)
height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )

input_width = 224
input_height = 224

y1 = int(height/3)
y2 = int(height/3 * 2)
x1 = int(width/3)
x2 = int(width/3 * 2)

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

tic = int(round(time.time() * 1000)) + 3000

print("Press [q] to quit") 
print("Press [p] to start/pause")
print("Press [d] to manually send command")

while(True):
    
    # Start video
    grabbed, frame = video_capture.read()

    cropped = frame[y1:y2, x1:x2]
    
    # display the cropped area
    rectangleColor              = (0,0,255)
    rectangleThickness          = 2
    cv2.rectangle(frame, (x1,y1), (x2,y2), rectangleColor, rectangleThickness)

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    if mode_status == MODE_START:
        
        #change the rectangle to green
        rectangleColor            = (50,175,50)
        cv2.rectangle(frame, (x1,y1), (x2,y2), rectangleColor, rectangleThickness)
        cv2.rectangle(frame, (0,0), (400,(label_count)*40), rectangleColor, -1) 
        
        image_array = np.asarray(imgRGB)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        
        # Load the image into the array
        data[0] = normalized_image_array
    
        # run the inference
        prediction = model.predict(data)

        # display predication on Image frame
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        fontScale              = 1
        fontColor              = (255,255,255)
        lineThickness          = 1

        scoreLabel = 0
        score = 0
        
        for x in range(0, label_count):
            # Put prediction on screen
            line = ('(%s) = %0.0f' % (labels[x], int(round(prediction[0][x]*100)))) + "%"
            cv2.putText(frame, line, (10, (x+1)*35), font, fontScale, fontColor, lineThickness)
            
            # find the highest perdiction
            if score < prediction[0][x]:
                scoreLabel = x
                score = prediction[0][x]
        
        
        # Every 3 seconds, send prediction to micro:bit
        now = int(round(time.time() * 1000))
        if now > tic and score > .95:
            if scoreLabel == 0:
                cmd = 'a'

            elif scoreLabel == 1:
                cmd = 'b'
                
            elif scoreLabel == 2:
                cmd = 'c'

            ser.write(cmd.encode())
            print('Auto Send:', labels[scoreLabel])
            tic = int(round(time.time() * 1000)) + 3000

    # show the output frame
    cv2.imshow("Frame", frame) 
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        print('Mode: Quit')
        break

    elif key == ord('p'):
        
        if mode_status != MODE_PAUSE:
            mode_status = MODE_PAUSE       
            print("Mode: Paused")


        elif mode_status == MODE_PAUSE:
            mode_status = MODE_START
            print ("Mode: Start")

    elif key == ord("d"):

        if mode_status == MODE_START:
            if scoreLabel == 0:
                cmd = 'a'

            elif scoreLabel == 1:
                cmd = 'b'
                
            elif scoreLabel == 2:
                cmd = 'c'

            ser.write(cmd.encode())
            print('Manual Send:', labels[scoreLabel])
            tic = int(round(time.time() * 1000)) + 3000
            
video_capture.release()
cv2.destroyAllWindows()


Press [q] to quit
Press [p] to start/pause
Press [d] to manually send command
Mode: Start
Auto Send: 2 none
Auto Send: 0 black
Auto Send: 2 none
Auto Send: 1 pink
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 2 none
Auto Send: 1 pink
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 0 black
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 2 none
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 0 black
Auto Send: 1 pi